In [276]:
#Importamos las librerias que utilizaremos
import pandas as pd
import numpy as np
import random
import csv

In [277]:
#Cargamos los datos de los archivos
data_ciu = pd.read_table('ciudades.tsv' , header = None)
data_tw = pd.read_table("tweets_100k.tsv", encoding = "utf-8" , error_bad_lines=False, quoting=csv.QUOTE_NONE)
data_ciu.columns = ["Ciudad" , "Latitud" , "Longitud" , "Prob"]

In [278]:
#Utilizamos la funcion multinomial y where para obtener una lista de ciudades que generaron tweets
N = 100000
probs= data_ciu["Prob"].tolist() #Se considero sumarle 0.001 mas a la probabilidad de Castro

_,m = np.where(np.random.multinomial(1, probs, size=(N))) #lista de ciudades por int

ciudades = []
for n in m:
    ciudades += [(data_ciu["Ciudad"].tolist())[n]] #lista de ciudades por string

In [279]:
#Definimos una funcion para obtener la cantidad de tweets por ciudad
def generar_pob(x):
    porc = []
    x = x.tolist()
    for i in range(10) :
        porc += [x.count(i)]
    return porc  

In [280]:
#Utilizamos la funcion multivariate_normal para generar puntos a partir de una lat y long central
latitudes = data_ciu["Latitud"].tolist()
longitudes = data_ciu["Longitud"].tolist()

cantidades = generar_pob(m) #Llamamos la funcion anterior
nuevas_c =[]
for i in cantidades :
    nuevas_c += [i/1000]
nuevas_lat = []
nuevas_lon = []
for i in m:
    cant = nuevas_c[i]
    mean = [latitudes[i],longitudes[i]] #Por cada tweet se llama la funcion segun la ciudad origen
    cov = [[cant*0.0003,0],[0,cant*0.0003]] #La probabilidad de dispersion de los tweets aumenta segun la cant de tweets
    x,y = np.random.multivariate_normal(mean, cov, 1).T
    nuevas_lat += x.tolist()
    nuevas_lon += y.tolist()
#BONUS: Se utilizo la funcion multivariate_normal

In [281]:
#Agregamos a cada fila del dataframe data_tw, una ciudad de origen con su respectiva lat y longitud generada
data_tw["ciudad"] = ciudades
data_tw["lat"] = nuevas_lat
data_tw["lon"] = nuevas_lon

In [282]:
#Guardamos la tabla (data_tw con modificaciones) en un archivo .csv (tweets_pos)
data_tw.to_csv("tweets_pos.csv",encoding='utf-8')

In [283]:
#Utilizamos la funcion uniforme para seleccionar 1000 tweets del dataframe anterior
a = 0
b = 99999
pts = np.random.uniform(a, b, 1000)

nueva = []
for pt in pts :
    nueva += [int(pt)]#Cambiamos a int para obtener las posiciones de los tweets seleccionados

In [284]:
#Guardamos los datos obtenidos en listas
tw_selecc = []
date_selecc = []
name_selecc = []
lat_selecc = []
lon_selecc = []
ciudad_selecc = []
for num in nueva :
    tw_selecc += [data_tw.loc[num]["text_tweet"]]
    date_selecc += [data_tw.loc[num]["creation_date"]]
    name_selecc += [data_tw.loc[num]["screen_name"]]
    ciudad_selecc += [data_tw.loc[num]["ciudad"]]
    lat_selecc += [data_tw.loc[num]["lat"]]
    lon_selecc += [data_tw.loc[num]["lon"]]

In [285]:
#Creamos un DataFrame con los datos obtenidos, a partir de un diccionario
dicc = {"screen_name" : name_selecc,
         "creation_date" : date_selecc,
         "text_tweet" : tw_selecc,
         "ciudad" : ciudad_selecc,
         "lat" : lat_selecc,
         "lon" : lon_selecc}


sample = pd.DataFrame.from_dict(dicc)
sample =  sample[['screen_name','creation_date','text_tweet','ciudad','lat','lon']] #Organizamos las columnas

In [286]:
#Guardamos la tabla (sample) en un archivo .csv (tweets_pos_sample)
sample.to_csv("tweets_pos_sample.csv",encoding='utf-8')
#10 seg de proceso aprox.